# Spam Email Classification using Transformer-Based Models with LoRA Fine-Tuning

## Overview
This notebook implements a production-ready spam email classification system using state-of-the-art transformer models with LoRA (Low-Rank Adaptation) fine-tuning. This implementation demonstrates an industry-standard approach that offers superior performance, efficiency, and deployment readiness.

## Why LoRA Fine-Tuning Instead of Building from Scratch?

### For All Readers (High-Level Explanation)

Instead of building a basic sequential neural network from scratch, this project uses **LoRA fine-tuning on pre-trained transformer models**. Here's why this approach makes more sense in real-world applications:

**Key Benefits:**
- **Faster Development & Training**: Fine-tuning takes hours instead of days, allowing for rapid prototyping and iteration
- **Superior Performance**: Pre-trained models have already learned language patterns from billions of text samples, giving us a head start
- **Production-Ready**: This approach is used by industry leaders (OpenAI, Google, Meta) for deploying ML models at scale
- **Cost-Effective**: Requires significantly less computational resources and training time compared to training from scratch
- **Better Generalization**: Pre-trained models handle diverse email patterns and evolving spam tactics more effectively

**Real-World Context:**
In a production environment, delivering a high-performing model quickly is critical. LoRA fine-tuning allows us to achieve 99%+ accuracy in a fraction of the time it would take to develop and tune a custom architecture, making it the preferred choice for time-sensitive deployments.

## Technical Deep Dive (Optional Reading)

### For Technical Readers: How LoRA Works Conceptually

**What is LoRA?**
LoRA (Low-Rank Adaptation) is a parameter-efficient fine-tuning technique that freezes the pre-trained model weights and injects trainable low-rank matrices into each layer. Instead of updating millions of parameters, we only train a small fraction (typically 0.1-1% of total parameters).

**How It Works:**
- Pre-trained transformer weights remain frozen
- Small trainable matrices (rank decomposition) are added to attention layers
- During fine-tuning, only these small matrices are updated
- The low-rank constraint acts as regularization, preventing overfitting

**Why This Matters for Spam Detection:**
- **Adaptability**: Spam tactics evolve constantly; LoRA allows quick retraining on new patterns
- **Memory Efficiency**: Multiple spam classifiers (different domains/languages) can share the same base model with different LoRA adapters
- **Deployment Flexibility**: Easy to update, version, and A/B test different adaptations

### Model Selection & Benchmarking Strategy

**Models Evaluated:**
1. **ELECTRA-base-discriminator**: Efficient pre-training approach, excellent for classification tasks
2. **RoBERTa-base**: Robust optimization of BERT, strong language understanding

**Why These Models?**
These models are the most commonly used base models for spam classification on HuggingFace, demonstrating proven effectiveness in production environments.

**Ablation Study:**
- Each model tested with LoRA rank 4 and rank 8
- Total configurations: 2 models × 2 ranks = 4 experiments
- This systematic comparison ensures we select the optimal configuration for accuracy and efficiency

## Notebook Structure

This notebook is organized into the following sections:

1. **Setup & Configuration** - Environment setup, library imports, and configuration loading
2. **Model Download** - Downloading pre-trained models for local use
3. **Data Loading & Exploration** - Loading the spam dataset and performing exploratory data analysis (EDA)
4. **Data Preprocessing & Splitting** - Text cleaning and stratified train/validation/test split
5. **Model Architecture & Training Setup** - LoRA configuration and model initialization
6. **Training Execution & Results** - Training loop with logging and checkpointing
7. **Model Evaluation & Comparison** - Comprehensive evaluation with multiple metrics and visualizations
8. **Conclusion & Key Findings** - Summary of results and model selection

Each section contains detailed code with outputs for complete reproducibility.

# 1. Setup & Configuration

This section handles environment setup, library imports, configuration loading, and seed initialization for reproducibility.

In [ ]:
import os
import sys
import yaml
import random
import time
import re
from pathlib import Path
from collections import Counter

import numpy as np
import pandas as pd
from wordcloud import WordCloud

import warnings
warnings.filterwarnings(action='ignore')

import seaborn as sns
sns.set_style('whitegrid')

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (15, 10)

from peft import (
    get_peft_model,
    LoraConfig,
    TaskType
)

from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix,
    roc_curve,
    auc,
    precision_recall_curve,
    classification_report
)

import nltk
from nltk import ngrams
from nltk.corpus import stopwords

import spacy
from spacy.cli import download as spacy_download

from tqdm.auto import tqdm
from loguru import logger

from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    get_cosine_schedule_with_warmup,
    set_seed
)

import torch
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
from torch.cuda.amp import autocast, GradScaler

In [ ]:
with open('config.yaml', 'r') as f:
    config = yaml.safe_load(f)

SEED = config['data']['random_seed']
DEVICE = config['device'] if torch.cuda.is_available() else 'cpu'

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
set_seed(SEED)

for path_key in config['paths'].values():
    Path(path_key).mkdir(parents=True, exist_ok=True)

log_file = Path(config['paths']['logs']) / 'training.log'
logger.remove()
logger.add(log_file, level=config['logging']['level'], format="{time:YYYY-MM-DD HH:mm:ss} | {level} | {message}")
logger.add(sys.stdout, level=config['logging']['level'], format="{time:HH:mm:ss} | {level} | {message}")

logger.info(f"Configuration loaded successfully")
logger.info(f"Device: {DEVICE}")
logger.info(f"Random seed: {SEED}")

23:30:29 | INFO | Configuration loaded successfully
23:30:29 | INFO | Device: cuda
23:30:29 | INFO | Random seed: 42


In [ ]:
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt', quiet=True)
    nltk.download('stopwords', quiet=True)
    nltk.download('wordnet', quiet=True)
    nltk.download('averaged_perceptron_tagger', quiet=True)
    logger.info("Downloaded NLTK data")

if config['spacy']['auto_download']:
    try:
        nlp = spacy.load(config['spacy']['model'])
        logger.info(f"Loaded spaCy model: {config['spacy']['model']}")
    except Exception as e:
        logger.warning(f"Error Logged: {str(e)}")
        logger.info(f"Downloading spaCy model: {config['spacy']['model']}")
        spacy_download(config['spacy']['model'])
        nlp = spacy.load(config['spacy']['model'])
        logger.info(f"Loaded spaCy model: {config['spacy']['model']}")

09:38:40 | INFO | Loaded spaCy model: en_core_web_sm


In [ ]:
def save_plot(filename: str, subdir: str = 'eda'):
    save_path = Path(config['paths']['visualizations']) / subdir
    save_path.mkdir(parents=True, exist_ok=True)
    full_path = save_path / filename
    plt.savefig(full_path, dpi=config['visualization']['dpi'], bbox_inches='tight')
    plt.close()
    logger.info(f"Saved plot: {full_path}")
    return str(full_path)

def save_dataframe_in_multiple_formats(df: pd.DataFrame, name: str, output_dir: str = './data_splits'):
    os.makedirs(output_dir, exist_ok=True)

    base_path = os.path.join(output_dir, name)
    csv_path = f"{base_path}.csv"
    parquet_path = f"{base_path}.parquet"
    feather_path = f"{base_path}.feather"

    print(f"\nSaving {name} split...")

    df.to_csv(csv_path, index=False)
    print(f"  -> Saved CSV to: {csv_path}")

    df.to_parquet(parquet_path, index=False)
    print(f"  -> Saved Parquet to: {parquet_path}")

    df.to_feather(feather_path)
    print(f"  -> Saved Feather to: {feather_path}")
    del df



# 2. Model Download (Optional - Run Once)

This section downloads the pre-trained transformer models from HuggingFace and saves them locally. This is optional if you want to work offline or avoid repeated downloads.

**Models to Download:**
- [`google/electra-base-discriminator`](https://huggingface.co/google/electra-base-discriminator)
- [`FacebookAI/roberta-base`](https://huggingface.co/FacebookAI/roberta-base)

**Note:** You can skip this cell if you want to download models automatically during training, or if you're loading models from a different location.

In [ ]:
import os
from transformers import AutoTokenizer, AutoModelForSequenceClassification

MODEL_NAMES = ["google/electra-base-discriminator", "FacebookAI/roberta-base"]
SAVE_DIR = "./local_models"

os.makedirs(SAVE_DIR, exist_ok=True)

local_paths = {}

for model_name in MODEL_NAMES:
    print(f"Downloading and saving: {model_name}")

    model_folder_name = model_name.replace("/", "_")
    model_dir = os.path.join(SAVE_DIR, model_folder_name)
    os.makedirs(model_dir, exist_ok=True)

    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

    tokenizer.save_pretrained(model_dir)
    model.save_pretrained(model_dir)

    local_paths[model_name] = model_dir

print("Download complete!")
print("\nLocal model directories:")
for k, v in local_paths.items():
    print(f"{k} : {v}")

# 3. Data Loading & Exploration (EDA)

This section loads the spam email dataset and performs comprehensive exploratory data analysis including:
- Dataset overview and statistics
- Class distribution analysis
- Text length analysis
- N-gram analysis (unigrams and bigrams)
- Word clouds visualization
- Special character and pattern analysis
- Linguistic analysis using spaCy (POS tags and named entities)

In [ ]:
logger.info("Loading dataset...")
df = pd.read_csv(config['data']['path'])
logger.info(f"Dataset loaded: {df.shape[0]} rows, {df.shape[1]} columns")

print("="*60)
print("DATASET OVERVIEW")
print("="*60)
print(f"Shape: {df.shape}")
print(f"\nColumns: {df.columns.tolist()}")
print(f"\nData types:\n{df.dtypes}")
print(f"\nMissing values:\n{df.isnull().sum()}")
print(f"\nDuplicate rows: {df.duplicated().sum()}")
print(f"Duplicate texts: {df[config['data']['text_column']].duplicated().sum()}")
print(f"\nLabel distribution:\n{df[config['data']['label_column']].value_counts()}")
print(f"\nSample data:")
print(df.head())

23:30:32 | INFO | Loading dataset...
23:30:33 | INFO | Dataset loaded: 83448 rows, 2 columns
DATASET OVERVIEW
Shape: (83448, 2)

Columns: ['label', 'text']

Data types:
label     int64
text     object
dtype: object

Missing values:
label    0
text     0
dtype: int64

Duplicate rows: 0
Duplicate texts: 2

Label distribution:
label
1    43910
0    39538
Name: count, dtype: int64

Sample data:
   label                                               text
0      1  ounce feather bowl hummingbird opec moment ala...
1      1  wulvob get your medircations online qnb ikud v...
2      0   computer connection from cnn com wednesday es...
3      1  university degree obtain a prosperous future m...
4      0  thanks for all your answers guys i know i shou...


In [ ]:
logger.info("Computing text statistics...")

df['text_length'] = df[config['data']['text_column']].str.len()
df['word_count'] = df[config['data']['text_column']].str.split().str.len()
df['avg_word_length'] = df.apply(lambda row: np.mean([len(word) for word in row[config['data']['text_column']].split()]) if row['word_count'] > 0 else 0, axis=1)
df['unique_words'] = df[config['data']['text_column']].apply(lambda x: len(set(x.lower().split())))

print("\n" + "="*60)
print("TEXT LENGTH STATISTICS")
print("="*60)
print(df[['text_length', 'word_count', 'avg_word_length', 'unique_words']].describe())

print("\n" + "="*60)
print("STATISTICS BY CLASS")
print("="*60)
for label in df[config['data']['label_column']].unique():
    label_name = 'SPAM' if label == 1 else 'HAM'
    print(f"\n{label_name} (label={label}):")
    print(df[df[config['data']['label_column']] == label][['text_length', 'word_count', 'avg_word_length', 'unique_words']].describe())

fig, axes = plt.subplots(2, 2, figsize=tuple(config['visualization']['figure_sizes']['large']))

axes[0, 0].hist([df[df[config['data']['label_column']]==1]['text_length'], df[df[config['data']['label_column']]==0]['text_length']],  label=['Spam', 'Ham'], bins=50, alpha=0.7)
axes[0, 0].set_xlabel('Text Length (characters)')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].set_title('Text Length Distribution')
axes[0, 0].legend()

axes[0, 1].hist([df[df[config['data']['label_column']]==1]['word_count'], df[df[config['data']['label_column']]==0]['word_count']], label=['Spam', 'Ham'], bins=50, alpha=0.7)
axes[0, 1].set_xlabel('Word Count')
axes[0, 1].set_ylabel('Frequency')
axes[0, 1].set_title('Word Count Distribution')
axes[0, 1].legend()

axes[1, 0].boxplot([df[df[config['data']['label_column']]==1]['avg_word_length'], df[df[config['data']['label_column']]==0]['avg_word_length']], labels=['Spam', 'Ham'])
axes[1, 0].set_ylabel('Average Word Length')
axes[1, 0].set_title('Average Word Length by Class')

axes[1, 1].hist([df[df[config['data']['label_column']]==1]['unique_words'], df[df[config['data']['label_column']]==0]['unique_words']], label=['Spam', 'Ham'], bins=50, alpha=0.7)
axes[1, 1].set_xlabel('Unique Words')
axes[1, 1].set_ylabel('Frequency')
axes[1, 1].set_title('Unique Words Distribution')
axes[1, 1].legend()

plt.tight_layout()
save_plot('text_statistics.png')

09:15:17 | INFO | Computing text statistics...

TEXT LENGTH STATISTICS
         text_length     word_count  avg_word_length  unique_words
count   83448.000000   83448.000000     83448.000000  83448.000000
mean     1662.952725     282.811775         4.918200    127.848193
std      4178.578068     724.818152         1.773719    145.888237
min         1.000000       1.000000         1.000000      1.000000
25%       449.000000      80.000000         4.000000     54.000000
50%       879.000000     152.000000         4.762500     92.000000
75%      1861.000000     312.000000         5.489655    156.000000
max    598705.000000  101984.000000       175.500000   5182.000000

STATISTICS BY CLASS

SPAM (label=1):
         text_length    word_count  avg_word_length  unique_words
count   43910.000000  43910.000000     43910.000000  43910.000000
mean     1249.887247    208.754634         5.149023    111.648463
std      1978.631507    338.269557         2.164387    107.537529
min         1.000000    

'visualizations/eda/text_statistics.png'

In [ ]:
logger.info("Analyzing token lengths with tokenizer...")

tokenizer_test = AutoTokenizer.from_pretrained(config['models']['names'][0])
df['token_length'] = df[config['data']['text_column']].apply(
    lambda x: len(tokenizer_test.encode(x, add_special_tokens=True, truncation=False))
)

print("\n" + "="*60)
print("TOKEN LENGTH STATISTICS")
print("="*60)
print(df['token_length'].describe())

print(f"\nTexts > 512 tokens: {(df['token_length'] > 512).sum()} ({(df['token_length'] > 512).sum() / len(df) * 100:.2f}%)")
print(f"Texts > 1024 tokens: {(df['token_length'] > 1024).sum()} ({(df['token_length'] > 1024).sum() / len(df) * 100:.2f}%)")
print(f"Texts > 2048 tokens: {(df['token_length'] > 2048).sum()} ({(df['token_length'] > 2048).sum() / len(df) * 100:.2f}%)")

for label in df[config['data']['label_column']].unique():
    label_name = 'SPAM' if label == 1 else 'HAM'
    print(f"\n{label_name} > 512 tokens: {(df[df[config['data']['label_column']]==label]['token_length'] > 512).sum()}")

fig, axes = plt.subplots(2, 2, figsize=tuple(config['visualization']['figure_sizes']['large']))

axes[0, 0].hist(df['token_length'].clip(upper=2000), bins=50, edgecolor='black', alpha=0.7)
axes[0, 0].axvline(x=512, color='red', linestyle='--', linewidth=2, label='512 token limit')
axes[0, 0].set_xlabel('Token Length (capped at 2000)')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].set_title('Overall Token Length Distribution')
axes[0, 0].legend()

spam_tokens = df[df[config['data']['label_column']]==1]['token_length']
ham_tokens = df[df[config['data']['label_column']]==0]['token_length']
axes[0, 1].hist([spam_tokens.clip(upper=2000), ham_tokens.clip(upper=2000)], bins=50, label=['Spam', 'Ham'], alpha=0.7, edgecolor='black')
axes[0, 1].axvline(x=512, color='red', linestyle='--', linewidth=2)
axes[0, 1].set_xlabel('Token Length (capped at 2000)')
axes[0, 1].set_ylabel('Frequency')
axes[0, 1].set_title('Token Length: Spam vs Ham')
axes[0, 1].legend()

axes[1, 0].boxplot([spam_tokens, ham_tokens], labels=['Spam', 'Ham'])
axes[1, 0].axhline(y=512, color='red', linestyle='--', linewidth=2)
axes[1, 0].set_ylabel('Token Length')
axes[1, 0].set_title('Token Length Distribution by Class')

axes[1, 1].hist(df['token_length'], bins=100, cumulative=True, density=True, alpha=0.7, edgecolor='black')
axes[1, 1].axvline(x=512, color='red', linestyle='--', linewidth=2)
axes[1, 1].axhline(y=0.95, color='green', linestyle='--', linewidth=1, alpha=0.5)
axes[1, 1].set_xlabel('Token Length')
axes[1, 1].set_ylabel('Cumulative Probability')
axes[1, 1].set_title('Cumulative Distribution')
axes[1, 1].set_xlim(0, 2000)

plt.tight_layout()
save_plot('token_length_analysis.png')

09:15:26 | INFO | Analyzing token lengths with DeBERTa tokenizer...


Token indices sequence length is longer than the specified maximum sequence length for this model (635 > 512). Running this sequence through the model will result in indexing errors



TOKEN LENGTH STATISTICS
count     83448.000000
mean        379.136888
std        1032.527373
min           3.000000
25%         105.000000
50%         204.000000
75%         423.000000
max      182763.000000
Name: token_length, dtype: float64

Texts > 512 tokens: 15831 (18.97%)
Texts > 1024 tokens: 4648 (5.57%)
Texts > 2048 tokens: 1306 (1.57%)

SPAM > 512 tokens: 6287

HAM > 512 tokens: 9544
09:16:23 | INFO | Saved plot: visualizations/eda/token_length_analysis.png


'visualizations/eda/token_length_analysis.png'

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=tuple(config['visualization']['figure_sizes']['large']))

label_counts = df[config['data']['label_column']].value_counts()
axes[0].bar(['Ham (0)', 'Spam (1)'], [label_counts[0], label_counts[1]], color=['#2D8E4D', '#E67E22'])
axes[0].set_ylabel('Count')
axes[0].set_title('Label Distribution (Counts)')
for i, v in enumerate([label_counts[0], label_counts[1]]):
    axes[0].text(i, v + 500, str(v), ha='center', va='bottom', fontsize=12, fontweight='bold')

axes[1].pie([label_counts[0], label_counts[1]], labels=['Ham (0)', 'Spam (1)'], autopct='%1.1f%%', colors=['#2D8E4D', '#E67E22'], startangle=90)
axes[1].set_title('Label Distribution (Percentage)')

plt.tight_layout()
save_plot('class_distribution.png')

09:17:06 | INFO | Saved plot: visualizations/eda/class_distribution.png


'visualizations/eda/class_distribution.png'

In [ ]:
logger.info("Performing N-gram analysis...")

stop_words = set(stopwords.words('english'))

def get_ngrams(texts, n=1, top_k=20):
    all_ngrams = []
    for text in texts:
        words = [word.lower() for word in text.split() if word.lower() not in stop_words and len(word) > 2]
        all_ngrams.extend(list(ngrams(words, n)))
    return Counter(all_ngrams).most_common(top_k)

spam_texts = df[df[config['data']['label_column']]==1][config['data']['text_column']].tolist()
ham_texts = df[df[config['data']['label_column']]==0][config['data']['text_column']].tolist()

spam_unigrams = get_ngrams(spam_texts, n=1, top_k=20)
ham_unigrams = get_ngrams(ham_texts, n=1, top_k=20)
spam_bigrams = get_ngrams(spam_texts, n=2, top_k=15)
ham_bigrams = get_ngrams(ham_texts, n=2, top_k=15)

print("\n" + "="*60)
print("TOP 20 UNIGRAMS")
print("="*60)
print("\nSPAM:")
for ngram, count in spam_unigrams:
    print(f"{ngram[0]}: {count}")
print("\nHAM:")
for ngram, count in ham_unigrams:
    print(f"{ngram[0]}: {count}")

print("\n" + "="*60)
print("TOP 15 BIGRAMS")
print("="*60)
print("\nSPAM:")
for ngram, count in spam_bigrams:
    print(f"{' '.join(ngram)}: {count}")
print("\nHAM:")
for ngram, count in ham_bigrams:
    print(f"{' '.join(ngram)}: {count}")

fig, axes = plt.subplots(2, 2, figsize=tuple(config['visualization']['figure_sizes']['large']))

spam_uni_words = [ngram[0][0] for ngram in spam_unigrams]
spam_uni_counts = [count for ngram, count in spam_unigrams]
axes[0, 0].barh(spam_uni_words[::-1], spam_uni_counts[::-1], color='#E67E22')
axes[0, 0].set_xlabel('Count')
axes[0, 0].set_title('Top 20 Unigrams - SPAM')

ham_uni_words = [ngram[0][0] for ngram in ham_unigrams]
ham_uni_counts = [count for ngram, count in ham_unigrams]
axes[0, 1].barh(ham_uni_words[::-1], ham_uni_counts[::-1], color='#2D8E4D')
axes[0, 1].set_xlabel('Count')
axes[0, 1].set_title('Top 20 Unigrams - HAM')

spam_bi_words = [' '.join(ngram[0]) for ngram in spam_bigrams]
spam_bi_counts = [count for ngram, count in spam_bigrams]
axes[1, 0].barh(spam_bi_words[::-1], spam_bi_counts[::-1], color='#E67E22')
axes[1, 0].set_xlabel('Count')
axes[1, 0].set_title('Top 15 Bigrams - SPAM')

ham_bi_words = [' '.join(ngram[0]) for ngram in ham_bigrams]
ham_bi_counts = [count for ngram, count in ham_bigrams]
axes[1, 1].barh(ham_bi_words[::-1], ham_bi_counts[::-1], color='#2D8E4D')
axes[1, 1].set_xlabel('Count')
axes[1, 1].set_title('Top 15 Bigrams - HAM')

plt.tight_layout()
save_plot('ngram_analysis.png')

09:17:08 | INFO | Performing N-gram analysis...

TOP 20 UNIGRAMS

SPAM:
escapenumber: 333179
escapelong: 188102
com: 29001
http: 27754
per: 26141
pills: 23128
escapenumbermg: 20541
price: 18723
company: 15859
one: 15853
save: 15005
may: 14662
item: 14472
time: 12351
please: 12330
get: 11986
new: 11385
money: 11378
information: 11158
see: 10819

HAM:
escapenumber: 798212
http: 54360
enron: 52856
org: 42530
com: 40658
escapelong: 38909
ect: 34743
help: 32512
samba: 30703
list: 28223
www: 28129
please: 26786
new: 25642
would: 25381
source: 25127
data: 21931
may: 21597
stat: 21220
ethz: 20493
html: 19548

TOP 15 BIGRAMS

SPAM:
escapelong escapelong: 156976
escapenumber escapenumber: 144250
escapenumber per: 24186
pills escapenumbermg: 19809
escapenumbermg escapenumber: 17712
escapenumber pills: 16672
per item: 13956
save escapenumber: 10386
price escapenumber: 9724
http www: 7147
item save: 7122
retail price: 7072
escapenumber adobe: 4804
escapenumber escapelong: 4297
low escapenumber: 378

'visualizations/eda/ngram_analysis.png'

In [ ]:
logger.info("Generating word clouds...")

spam_text = ' '.join(spam_texts)
ham_text = ' '.join(ham_texts)

fig, axes = plt.subplots(1, 2, figsize=tuple(config['visualization']['figure_sizes']['large']))

wc_spam = WordCloud(width=800, height=400, background_color='white', colormap='Reds', stopwords=stop_words, max_words=100).generate(spam_text)
axes[0].imshow(wc_spam, interpolation='bilinear')
axes[0].set_title('Word Cloud - SPAM', fontsize=16, fontweight='bold')
axes[0].axis('off')

wc_ham = WordCloud(width=800, height=400, background_color='white', colormap='Greens', stopwords=stop_words, max_words=100).generate(ham_text)
axes[1].imshow(wc_ham, interpolation='bilinear')
axes[1].set_title('Word Cloud - HAM', fontsize=16, fontweight='bold')
axes[1].axis('off')

plt.tight_layout()
save_plot('wordclouds.png')

09:17:29 | INFO | Generating word clouds...
09:18:10 | INFO | Saved plot: visualizations/eda/wordclouds.png


'visualizations/eda/wordclouds.png'

In [ ]:
logger.info("Analyzing special characters and patterns...")

df['has_url'] = df[config['data']['text_column']].str.contains(r'http[s]?://|www\.', regex=True, case=False).astype(int)
df['has_email'] = df[config['data']['text_column']].str.contains(r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}', regex=True).astype(int)
df['has_phone'] = df[config['data']['text_column']].str.contains(r'\b\d{3}[-.]?\d{3}[-.]?\d{4}\b', regex=True).astype(int)
df['dollar_count'] = df[config['data']['text_column']].str.count(r'\$')
df['percent_count'] = df[config['data']['text_column']].str.count(r'%')
df['exclamation_count'] = df[config['data']['text_column']].str.count(r'!')
df['caps_ratio'] = df[config['data']['text_column']].apply(lambda x: sum(1 for c in x if c.isupper()) / len(x) if len(x) > 0 else 0)
df['punctuation_density'] = df[config['data']['text_column']].apply(lambda x: sum(1 for c in x if c in '!@#$%^&*()') / len(x) if len(x) > 0 else 0)
df['has_html'] = df[config['data']['text_column']].str.contains(r'<[^>]+>', regex=True).astype(int)

special_char_features = ['has_url', 'has_email', 'has_phone', 'dollar_count', 'percent_count', 'exclamation_count', 'caps_ratio', 'punctuation_density', 'has_html']

print("\n" + "="*60)
print("SPECIAL CHARACTER ANALYSIS BY CLASS")
print("="*60)
for label in [0, 1]:
    label_name = 'HAM' if label == 0 else 'SPAM'
    print(f"\n{label_name} (label={label}):")
    print(df[df[config['data']['label_column']]==label][special_char_features].describe())

fig, axes = plt.subplots(3, 3, figsize=tuple(config['visualization']['figure_sizes']['large']))
axes = axes.flatten()

for idx, feature in enumerate(special_char_features):
    spam_vals = df[df[config['data']['label_column']]==1][feature]
    ham_vals = df[df[config['data']['label_column']]==0][feature]
    axes[idx].hist([spam_vals, ham_vals], label=['Spam', 'Ham'], bins=30, alpha=0.7)
    axes[idx].set_xlabel(feature.replace('_', ' ').title())
    axes[idx].set_ylabel('Frequency')
    axes[idx].set_title(f'{feature.replace("_", " ").title()} Distribution')
    axes[idx].legend()

plt.tight_layout()
save_plot('special_character_analysis.png')

09:18:10 | INFO | Analyzing special characters and patterns...

SPECIAL CHARACTER ANALYSIS BY CLASS

HAM (label=0):
       has_url  has_email     has_phone  dollar_count  percent_count  \
count  39538.0    39538.0  39538.000000  39538.000000   39538.000000   
mean       0.0        0.0      0.019273      0.414917       0.088851   
std        0.0        0.0      0.137483      5.409788       1.111040   
min        0.0        0.0      0.000000      0.000000       0.000000   
25%        0.0        0.0      0.000000      0.000000       0.000000   
50%        0.0        0.0      0.000000      0.000000       0.000000   
75%        0.0        0.0      0.000000      0.000000       0.000000   
max        0.0        0.0      1.000000    349.000000      90.000000   

       exclamation_count  caps_ratio  punctuation_density  has_html  
count       39538.000000     39538.0         39538.000000   39538.0  
mean            0.251758         0.0             0.002593       0.0  
std             2.285224 

'visualizations/eda/special_character_analysis.png'

In [ ]:
logger.info("Performing linguistic analysis with spaCy...")

def analyze_with_spacy(texts, sample_size=1000):
    sampled_texts = random.sample(texts, min(sample_size, len(texts)))

    entities = []
    pos_tags = []

    for text in tqdm(sampled_texts, desc="Processing with spaCy"):
        doc = nlp(text[:1000])
        entities.extend([ent.label_ for ent in doc.ents])
        pos_tags.extend([token.pos_ for token in doc])

    return Counter(entities), Counter(pos_tags)

spam_entities, spam_pos = analyze_with_spacy(spam_texts)
ham_entities, ham_pos = analyze_with_spacy(ham_texts)

print("\n" + "="*60)
print("TOP NAMED ENTITIES")
print("="*60)
print("\nSPAM:")
for ent, count in spam_entities.most_common(10):
    print(f"{ent}: {count}")
print("\nHAM:")
for ent, count in ham_entities.most_common(10):
    print(f"{ent}: {count}")

print("\n" + "="*60)
print("TOP POS TAGS")
print("="*60)
print("\nSPAM:")
for pos, count in spam_pos.most_common(10):
    print(f"{pos}: {count}")
print("\nHAM:")
for pos, count in ham_pos.most_common(10):
    print(f"{pos}: {count}")

fig, axes = plt.subplots(2, 2, figsize=tuple(config['visualization']['figure_sizes']['large']))

spam_ent_labels = [ent for ent, _ in spam_entities.most_common(10)]
spam_ent_counts = [count for _, count in spam_entities.most_common(10)]
axes[0, 0].barh(spam_ent_labels[::-1], spam_ent_counts[::-1], color='#E67E22')
axes[0, 0].set_xlabel('Count')
axes[0, 0].set_title('Top Named Entities - SPAM')

ham_ent_labels = [ent for ent, _ in ham_entities.most_common(10)]
ham_ent_counts = [count for _, count in ham_entities.most_common(10)]
axes[0, 1].barh(ham_ent_labels[::-1], ham_ent_counts[::-1], color='#2D8E4D')
axes[0, 1].set_xlabel('Count')
axes[0, 1].set_title('Top Named Entities - HAM')

spam_pos_labels = [pos for pos, _ in spam_pos.most_common(10)]
spam_pos_counts = [count for _, count in spam_pos.most_common(10)]
axes[1, 0].bar(spam_pos_labels, spam_pos_counts, color='#E67E22')
axes[1, 0].set_xlabel('POS Tag')
axes[1, 0].set_ylabel('Count')
axes[1, 0].set_title('Top POS Tags - SPAM')
axes[1, 0].tick_params(axis='x', rotation=45)

ham_pos_labels = [pos for pos, _ in ham_pos.most_common(10)]
ham_pos_counts = [count for _, count in ham_pos.most_common(10)]
axes[1, 1].bar(ham_pos_labels, ham_pos_counts, color='#2D8E4D')
axes[1, 1].set_xlabel('POS Tag')
axes[1, 1].set_ylabel('Count')
axes[1, 1].set_title('Top POS Tags - HAM')
axes[1, 1].tick_params(axis='x', rotation=45)

plt.tight_layout()
save_plot('linguistic_analysis.png')

09:18:29 | INFO | Performing linguistic analysis with spaCy...


Processing with spaCy: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:24<00:00, 40.77it/s]



TOP NAMED ENTITIES

SPAM:
CARDINAL: 1486
PERSON: 966
ORG: 823
DATE: 775
MONEY: 430
GPE: 360
NORP: 226
TIME: 140
PERCENT: 98
ORDINAL: 82

HAM:
PERSON: 2316
CARDINAL: 1470
ORG: 1460
DATE: 1448
GPE: 513
TIME: 350
NORP: 201
MONEY: 148
ORDINAL: 118
QUANTITY: 65

TOP POS TAGS

SPAM:
NOUN: 29120
PROPN: 13005
VERB: 12028
PUNCT: 9133
ADJ: 8753
ADP: 8587
PRON: 8128
DET: 6257
AUX: 4717
SPACE: 4599

HAM:
NOUN: 33178
PROPN: 21545
VERB: 14654
PUNCT: 13691
ADP: 10864
PRON: 8083
DET: 8078
ADJ: 7607
AUX: 6274
SPACE: 5033
09:19:15 | INFO | Saved plot: visualizations/eda/linguistic_analysis.png


'visualizations/eda/linguistic_analysis.png'

# 4. Data Preprocessing & Splitting

This section prepares the data for training by:
- Cleaning and preprocessing text (if needed)
- Performing stratified train/validation/test split (80/10/10)
- Ensuring balanced class distribution across all splits
- Saving splits in multiple formats (CSV, Parquet, Feather)

**Design Choice: Stratified Split**
We use stratified splitting to ensure equal class distribution (spam vs ham) across train, validation, and test sets. This is critical for:
- Preventing class imbalance in any split
- Ensuring fair evaluation metrics
- Maintaining consistent model performance across different data subsets

In [ ]:
logger.info("Starting data preprocessing...")

df_clean = df[[config['data']['text_column'], config['data']['label_column']]].copy()
logger.info(f"Dataset shape before preprocessing: {df_clean.shape}")

logger.info("Dataset is clean, no preprocessing needed")
logger.info(f"Final dataset shape: {df_clean.shape}")

logger.info("Splitting dataset into train/val/test...")

train_df, temp_df = train_test_split(
    df_clean,
    test_size=config['data']['test_size'] + config['data']['val_size'],
    random_state=config['data']['random_seed'],
    stratify=df_clean[config['data']['label_column']]
)

val_df, test_df = train_test_split(
    temp_df,
    test_size=config['data']['test_size'] / (config['data']['test_size'] + config['data']['val_size']),
    random_state=config['data']['random_seed'],
    stratify=temp_df[config['data']['label_column']]
)

logger.info(f"Train set: {len(train_df)} samples")
logger.info(f"Validation set: {len(val_df)} samples")
logger.info(f"Test set: {len(test_df)} samples")

print("\n" + "="*60)
print("DATASET SPLIT SUMMARY")
print("="*60)
print(f"Train: {len(train_df)} ({len(train_df)/len(df_clean)*100:.1f}%)")
print(f"Val:   {len(val_df)} ({len(val_df)/len(df_clean)*100:.1f}%)")
print(f"Test:  {len(test_df)} ({len(test_df)/len(df_clean)*100:.1f}%)")

print("\nLabel distribution in splits:")
print(f"Train - Spam: {train_df[config['data']['label_column']].sum()} ({train_df[config['data']['label_column']].sum()/len(train_df)*100:.1f}%)")
print(f"Val   - Spam: {val_df[config['data']['label_column']].sum()} ({val_df[config['data']['label_column']].sum()/len(val_df)*100:.1f}%)")
print(f"Test  - Spam: {test_df[config['data']['label_column']].sum()} ({test_df[config['data']['label_column']].sum()/len(test_df)*100:.1f}%)")

23:30:45 | INFO | Starting data preprocessing...
23:30:45 | INFO | Dataset shape before preprocessing: (83448, 2)
23:30:45 | INFO | Dataset is clean, no preprocessing needed
23:30:45 | INFO | Final dataset shape: (83448, 2)
23:30:45 | INFO | Splitting dataset into train/val/test...
23:30:45 | INFO | Train set: 66758 samples
23:30:45 | INFO | Validation set: 8345 samples
23:30:45 | INFO | Test set: 8345 samples

DATASET SPLIT SUMMARY
Train: 66758 (80.0%)
Val:   8345 (10.0%)
Test:  8345 (10.0%)

Label distribution in splits:
Train - Spam: 35128 (52.6%)
Val   - Spam: 4391 (52.6%)
Test  - Spam: 4391 (52.6%)


In [ ]:
save_dataframe_in_multiple_formats(train_df, 'train')
save_dataframe_in_multiple_formats(val_df, 'val')
save_dataframe_in_multiple_formats(test_df, 'test')


Saving train split...
  -> Saved CSV to: ./data_splits/train.csv
  -> Saved Parquet to: ./data_splits/train.parquet
  -> Saved Feather to: ./data_splits/train.feather

Saving val split...
  -> Saved CSV to: ./data_splits/val.csv
  -> Saved Parquet to: ./data_splits/val.parquet
  -> Saved Feather to: ./data_splits/val.feather

Saving test split...
  -> Saved CSV to: ./data_splits/test.csv
  -> Saved Parquet to: ./data_splits/test.parquet
  -> Saved Feather to: ./data_splits/test.feather


# 5. Model Architecture & Training Setup

This section defines:
- Custom PyTorch Dataset class for tokenization and data loading
- Training and evaluation functions with mixed precision
- LoRA (Low-Rank Adaptation) configuration
- Model training loop with early stopping

**Design Choices:**

**Why These Models?**
- **ELECTRA-base-discriminator**: Efficient pre-training approach optimized for classification tasks
- **RoBERTa-base**: Robust optimization of BERT with proven performance on text classification
- Both models are the most commonly used base models for spam classification on HuggingFace

**Hyperparameter Rationale:**
- **2 Epochs**: Sufficient for fine-tuning pre-trained models; prevents overfitting while allowing adaptation to spam patterns
- **Learning Rate (2e-4)**: Standard for LoRA fine-tuning, balances convergence speed and stability
- **Batch Size (16)**: Optimized for GPU memory efficiency with automatic reduction if OOM occurs
- **LoRA Ranks (4 and 8)**: Ablation study to find optimal parameter efficiency vs performance trade-off

**Why Precision Matters for Spam Detection:**
Precision is prioritized because:
- False positives (legitimate emails marked as spam) are more costly than false negatives
- Spam tactics evolve constantly; new patterns may not match training data exactly
- High precision ensures user trust in the classification system

In [ ]:
class SpamDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]

        encoding = self.tokenizer(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

def train_epoch(model, dataloader, optimizer, scheduler, scaler, device, tb_writer, global_step):
    model.train()
    total_loss = 0
    predictions = []
    true_labels = []

    progress_bar = tqdm(dataloader, desc="Training")

    for batch_idx, batch in enumerate(progress_bar):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()

        with autocast():
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss

        scaler.scale(loss).backward()
        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), config['training']['gradient_clipping'])
        scaler.step(optimizer)
        scaler.update()
        scheduler.step()

        total_loss += loss.item()

        preds = torch.argmax(outputs.logits, dim=1)
        predictions.extend(preds.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

        tb_writer.add_scalar('Train/BatchLoss', loss.item(), global_step)
        tb_writer.add_scalar('Train/LearningRate', scheduler.get_last_lr()[0], global_step)

        progress_bar.set_postfix({'loss': loss.item(), 'lr': scheduler.get_last_lr()[0]})
        global_step += 1

    avg_loss = total_loss / len(dataloader)
    accuracy = accuracy_score(true_labels, predictions)
    precision = precision_score(true_labels, predictions, zero_division=0)
    recall = recall_score(true_labels, predictions, zero_division=0)
    f1 = f1_score(true_labels, predictions, zero_division=0)

    return avg_loss, accuracy, precision, recall, f1, global_step

def evaluate(model, dataloader, device):
    model.eval()
    total_loss = 0
    predictions = []
    true_labels = []
    probs = []

    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Evaluating"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            with autocast():
                outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)

            total_loss += outputs.loss.item()

            logits = outputs.logits
            preds = torch.argmax(logits, dim=1)
            predictions.extend(preds.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())
            probs.extend(torch.softmax(logits, dim=1)[:, 1].cpu().numpy())

    avg_loss = total_loss / len(dataloader)
    accuracy = accuracy_score(true_labels, predictions)
    precision = precision_score(true_labels, predictions, zero_division=0)
    recall = recall_score(true_labels, predictions, zero_division=0)
    f1 = f1_score(true_labels, predictions, zero_division=0)

    return avg_loss, accuracy, precision, recall, f1, predictions, true_labels, probs

In [ ]:
def train_model(model_name, lora_rank, train_loader, val_loader, device):
    model_short_name = model_name.split('/')[-1]
    run_name = f"{model_short_name}_r{lora_rank}"

    logger.info(f"Starting training: {run_name}")

    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(
        model_name,
        num_labels=2,
        problem_type="single_label_classification"
    )

    lora_alpha = lora_rank * 2 if config['lora']['alpha'] == 'auto' else config['lora']['alpha']

    peft_config = LoraConfig(
        task_type=TaskType.SEQ_CLS,
        r=lora_rank,
        lora_alpha=lora_alpha,
        lora_dropout=config['lora']['dropout'],
        target_modules=config['lora']['target_modules'],
        bias="none"
    )

    model = get_peft_model(model, peft_config)
    model.print_trainable_parameters()
    model = model.to(device)

    logger.info(f"Model loaded: {model_name} with LoRA rank={lora_rank}, alpha={lora_alpha}")

    optimizer = torch.optim.AdamW(
        model.parameters(),
        lr=float(config['training']['learning_rate']),
        weight_decay=config['training']['weight_decay']
    )

    total_steps = len(train_loader) * config['training']['epochs']
    warmup_steps = int(total_steps * config['training']['warmup_ratio'])

    scheduler = get_cosine_schedule_with_warmup(
        optimizer,
        num_warmup_steps=warmup_steps,
        num_training_steps=total_steps
    )

    scaler = GradScaler()

    tb_writer = SummaryWriter(log_dir=str(Path(config['paths']['tensorboard']) / run_name))

    best_val_loss = float('inf')
    patience_counter = 0
    global_step = 0
    history = []

    start_time = time.time()

    for epoch in range(config['training']['epochs']):
        logger.info(f"Epoch {epoch+1}/{config['training']['epochs']}")

        train_loss, train_acc, train_prec, train_rec, train_f1, global_step = train_epoch(
            model, train_loader, optimizer, scheduler, scaler, device, tb_writer, global_step
        )

        val_loss, val_acc, val_prec, val_rec, val_f1, _, _, _ = evaluate(model, val_loader, device)

        logger.info(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | Train F1: {train_f1:.4f}")
        logger.info(f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f} | Val F1: {val_f1:.4f}")

        tb_writer.add_scalar('Train/EpochLoss', train_loss, epoch)
        tb_writer.add_scalar('Train/Accuracy', train_acc, epoch)
        tb_writer.add_scalar('Train/Precision', train_prec, epoch)
        tb_writer.add_scalar('Train/Recall', train_rec, epoch)
        tb_writer.add_scalar('Train/F1', train_f1, epoch)

        tb_writer.add_scalar('Val/Loss', val_loss, epoch)
        tb_writer.add_scalar('Val/Accuracy', val_acc, epoch)
        tb_writer.add_scalar('Val/Precision', val_prec, epoch)
        tb_writer.add_scalar('Val/Recall', val_rec, epoch)
        tb_writer.add_scalar('Val/F1', val_f1, epoch)

        history.append({
            'epoch': epoch + 1,
            'train_loss': train_loss,
            'train_acc': train_acc,
            'train_prec': train_prec,
            'train_rec': train_rec,
            'train_f1': train_f1,
            'val_loss': val_loss,
            'val_acc': val_acc,
            'val_prec': val_prec,
            'val_rec': val_rec,
            'val_f1': val_f1
        })

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0

            save_path = Path(config['paths']['models']) / run_name
            save_path.mkdir(parents=True, exist_ok=True)
            checkpoint_name = f"{run_name}_epoch{epoch+1}_val_loss_{val_loss:.4f}"
            model.save_pretrained(save_path / checkpoint_name)
            tokenizer.save_pretrained(save_path / checkpoint_name)

            logger.info(f"Saved best model: {checkpoint_name}")
        else:
            patience_counter += 1
            logger.info(f"No improvement. Patience: {patience_counter}/{config['early_stopping']['patience']}")

            if patience_counter >= config['early_stopping']['patience']:
                logger.info(f"Early stopping triggered at epoch {epoch+1}")
                break

    training_time = time.time() - start_time
    logger.info(f"Training completed for {run_name} in {training_time/60:.2f} minutes")

    tb_writer.close()

    history_df = pd.DataFrame(history)
    history_df.to_csv(Path(config['paths']['results']) / f"{run_name}_history.csv", index=False)

    return model, tokenizer, history_df, training_time

# 6. Training Execution & Results

This section trains all model configurations:
- 2 base models (ELECTRA, RoBERTa)
- 2 LoRA ranks (r=4, r=8)
- Total: 4 experiments

Each model is trained with:
- Early stopping (patience=2 was initially used when the **total epochs were 5**, but since the **total epochs were later reduced to 2**, early stopping is now **optional**.
- Mixed precision training for efficiency
- Gradient clipping for stability
- TensorBoard logging for monitoring
- Automatic checkpoint saving for best models

In [ ]:
logger.info("Starting model training loop...")

all_results = {}
training_times = {}

for model_name in config['models']['names']:
    for lora_rank in config['lora']['ranks']:
        run_name = f"{model_name.split('/')[-1]}_r{lora_rank}"
        logger.info("="*60)
        logger.info(f"Training: {run_name}")
        logger.info("="*60)

        try:
            tokenizer = AutoTokenizer.from_pretrained(model_name)

            train_dataset = SpamDataset(
                train_df[config['data']['text_column']].values,
                train_df[config['data']['label_column']].values,
                tokenizer,
                config['models']['max_length']
            )

            val_dataset = SpamDataset(
                val_df[config['data']['text_column']].values,
                val_df[config['data']['label_column']].values,
                tokenizer,
                config['models']['max_length']
            )

            train_loader = DataLoader(
                train_dataset,
                batch_size=config['training']['batch_size'],
                shuffle=True,
                num_workers=0
            )

            val_loader = DataLoader(
                val_dataset,
                batch_size=config['training']['batch_size'],
                shuffle=False,
                num_workers=0
            )

            model, tokenizer, history, training_time = train_model(
                model_name,
                lora_rank,
                train_loader,
                val_loader,
                DEVICE
            )

            all_results[run_name] = {
                'model': model,
                'tokenizer': tokenizer,
                'history': history
            }
            training_times[run_name] = training_time

            logger.info(f"Successfully completed training for {run_name}")

        except RuntimeError as e:
            if "out of memory" in str(e) and config['training']['auto_reduce_batch_size']:
                logger.warning(f"OOM error for {run_name}. Attempting with reduced batch size...")
                torch.cuda.empty_cache()

                reduced_batch_size = max(
                    config['training']['batch_size'] // 2,
                    config['training']['min_batch_size']
                )

                if reduced_batch_size >= config['training']['min_batch_size']:
                    logger.info(f"Retrying with batch_size={reduced_batch_size}")
                    train_loader = DataLoader(
                        train_dataset,
                        batch_size=reduced_batch_size,
                        shuffle=True,
                        num_workers=0
                    )
                    val_loader = DataLoader(
                        val_dataset,
                        batch_size=reduced_batch_size,
                        shuffle=False,
                        num_workers=0
                    )

                    model, tokenizer, history, training_time = train_model(
                        model_name,
                        lora_rank,
                        train_loader,
                        val_loader,
                        DEVICE
                    )

                    all_results[run_name] = {
                        'model': model,
                        'tokenizer': tokenizer,
                        'history': history
                    }
                    training_times[run_name] = training_time
                else:
                    logger.error(f"Cannot reduce batch size below minimum for {run_name}")
                    raise
            else:
                logger.error(f"Training failed for {run_name}: {str(e)}")
                raise

logger.info("="*60)
logger.info("All model training completed!")
logger.info("="*60)

23:30:48 | INFO | Starting model training loop...
23:30:48 | INFO | ============================================================
23:30:48 | INFO | Training: google_electra-base-discriminator_r4
23:30:48 | INFO | ============================================================
23:30:48 | INFO | Starting training: google_electra-base-discriminator_r4
trainable params: 1,255,682 || all params: 110,739,460 || trainable%: 1.1339
23:30:49 | INFO | Model loaded: ./local_models/google_electra-base-discriminator with LoRA rank=4, alpha=8
23:30:49 | INFO | Epoch 1/2


Evaluating: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 522/522 [03:37<00:00,  2.40it/s]

00:41:05 | INFO | Train Loss: 0.0882 | Train Acc: 0.9670 | Train F1: 0.9687
00:41:05 | INFO | Val Loss: 0.0296 | Val Acc: 0.9921 | Val F1: 0.9925
00:41:05 | INFO | Saved best model: google_electra-base-discriminator_r4_epoch1_val_loss_0.0296
00:41:05 | INFO | Epoch 2/2



Evaluating: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 522/522 [03:37<00:00,  2.40it/s]

01:51:21 | INFO | Train Loss: 0.0228 | Train Acc: 0.9942 | Train F1: 0.9945
01:51:21 | INFO | Val Loss: 0.0304 | Val Acc: 0.9921 | Val F1: 0.9925
01:51:21 | INFO | No improvement. Patience: 1/2
01:51:21 | INFO | Training completed for google_electra-base-discriminator_r4 in 140.54 minutes
01:51:21 | INFO | Successfully completed training for google_electra-base-discriminator_r4
01:51:21 | INFO | ============================================================
01:51:21 | INFO | Training: google_electra-base-discriminator_r8
01:51:21 | INFO | ============================================================
01:51:21 | INFO | Starting training: google_electra-base-discriminator_r8


...

trainable params: 1,919,234 || all params: 126,566,404 || trainable%: 1.5164
13:16:14 | INFO | Model loaded: ./local_models/FacebookAI_roberta-base with LoRA rank=8, alpha=16
13:16:14 | INFO | Epoch 1/2


Evaluating: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 522/522 [09:23<00:00,  1.08s/it]

17:48:41 | INFO | Train Loss: 0.0888 | Train Acc: 0.9668 | Train F1: 0.9686
17:48:41 | INFO | Val Loss: 0.0340 | Val Acc: 0.9914 | Val F1: 0.9918
17:48:41 | INFO | Saved best model: FacebookAI_roberta-base_r8_epoch1_val_loss_0.0340
17:48:41 | INFO | Epoch 2/2



Evaluating: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 522/522 [09:23<00:00,  1.08s/it]

22:21:09 | INFO | Train Loss: 0.0203 | Train Acc: 0.9949 | Train F1: 0.9952
22:21:09 | INFO | Val Loss: 0.0301 | Val Acc: 0.9930 | Val F1: 0.9934
22:21:09 | INFO | Saved best model: FacebookAI_roberta-base_r8_epoch2_val_loss_0.0301
22:21:09 | INFO | Training completed for FacebookAI_roberta-base_r8 in 544.92 minutes
22:21:09 | INFO | Successfully completed training for FacebookAI_roberta-base_r8
22:21:09 | INFO | ============================================================
22:21:09 | INFO | All model training completed!
22:21:09 | INFO | ============================================================


In [ ]:
all_results

{'google_electra-base-discriminator_r4': {'model': PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): ElectraForSequenceClassification(
      (electra): ElectraModel(
        (embeddings): ElectraEmbeddings(
          (word_embeddings): Embedding(30522, 768, padding_idx=0)
          (position_embeddings): Embedding(512, 768)
          (token_type_embeddings): Embedding(2, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): ElectraEncoder(
          (layer): ModuleList(
            (0-11): 12 x ElectraLayer(
              (attention): ElectraAttention(
                (self): ElectraSelfAttention(
                  (query): lora.Linear(
                    (base_layer): Linear(in_features=768, out_features=768, bias=True)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
 

# 7. Model Evaluation & Comparison

This section performs comprehensive evaluation on the test set:
- **Core Metrics**: Accuracy, Precision, Recall, F1-Score
- **Probabilistic Metrics**: ROC-AUC, PR-AUC
- **Confusion Matrices**: Detailed error analysis for each model
- **Classification Reports**: Per-class performance breakdown
- **Comparative Visualizations**:
  - Metrics comparison across all models
  - ROC curves comparison
  - Precision-Recall curves comparison
- **LoRA Ablation Study**: Analysis of rank 4 vs rank 8 performance and parameter efficiency

In [ ]:
logger.info("Evaluating all models on test set...")

evaluation_results = []

for run_name, result in all_results.items():
    logger.info(f"Evaluating {run_name}...")

    model = result['model']
    tokenizer = result['tokenizer']

    test_dataset = SpamDataset(
        test_df[config['data']['text_column']].values,
        test_df[config['data']['label_column']].values,
        tokenizer,
        config['models']['max_length']
    )

    test_loader = DataLoader(
        test_dataset,
        batch_size=config['training']['batch_size'],
        shuffle=False,
        num_workers=0
    )

    test_loss, test_acc, test_prec, test_rec, test_f1, predictions, true_labels, probs = evaluate(
        model, test_loader, DEVICE
    )

    fpr, tpr, _ = roc_curve(true_labels, probs)
    roc_auc = auc(fpr, tpr)

    precision_curve, recall_curve, _ = precision_recall_curve(true_labels, probs)
    pr_auc = auc(recall_curve, precision_curve)

    cm = confusion_matrix(true_labels, predictions)

    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    total_params = sum(p.numel() for p in model.parameters())

    evaluation_results.append({
        'model': run_name,
        'accuracy': test_acc,
        'precision': test_prec,
        'recall': test_rec,
        'f1_score': test_f1,
        'roc_auc': roc_auc,
        'pr_auc': pr_auc,
        'training_time_min': training_times[run_name] / 60,
        'trainable_params': trainable_params,
        'total_params': total_params,
        'predictions': predictions,
        'true_labels': true_labels,
        'probs': probs,
        'confusion_matrix': cm,
        'fpr': fpr,
        'tpr': tpr,
        'precision_curve': precision_curve,
        'recall_curve': recall_curve
    })

    logger.info(f"{run_name} - Acc: {test_acc:.4f}, Prec: {test_prec:.4f}, Rec: {test_rec:.4f}, F1: {test_f1:.4f}, ROC-AUC: {roc_auc:.4f}")

results_df = pd.DataFrame([{
    'model': r['model'],
    'accuracy': r['accuracy'],
    'precision': r['precision'],
    'recall': r['recall'],
    'f1_score': r['f1_score'],
    'roc_auc': r['roc_auc'],
    'pr_auc': r['pr_auc'],
    'training_time_min': r['training_time_min'],
    'trainable_params': r['trainable_params'],
    'total_params': r['total_params']
} for r in evaluation_results])

results_df.to_csv(Path(config['paths']['results']) / 'model_comparison.csv', index=False)
logger.info("Saved evaluation results to model_comparison.csv")

print("\n" + "="*80)
print("MODEL COMPARISON - TEST SET RESULTS")
print("="*80)
print(results_df.to_string(index=False))

22:21:57 | INFO | Evaluating all models on test set...
22:21:57 | INFO | Evaluating google_electra-base-discriminator_r4...


Evaluating: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 522/522 [03:38<00:00,  2.39it/s]

22:25:35 | INFO | google_electra-base-discriminator_r4 - Acc: 0.9939, Prec: 0.9945, Rec: 0.9939, F1: 0.9942, ROC-AUC: 0.9993
22:25:35 | INFO | Evaluating google_electra-base-discriminator_r8...



Evaluating: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 522/522 [03:37<00:00,  2.40it/s]

22:29:13 | INFO | google_electra-base-discriminator_r8 - Acc: 0.9935, Prec: 0.9952, Rec: 0.9925, F1: 0.9938, ROC-AUC: 0.9993
22:29:13 | INFO | Evaluating FacebookAI_roberta-base_r4...



Evaluating: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 522/522 [09:23<00:00,  1.08s/it]

22:38:36 | INFO | FacebookAI_roberta-base_r4 - Acc: 0.9941, Prec: 0.9950, Rec: 0.9939, F1: 0.9944, ROC-AUC: 0.9990
22:38:36 | INFO | Evaluating FacebookAI_roberta-base_r8...



Evaluating: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 522/522 [09:23<00:00,  1.08s/it]

22:48:00 | INFO | FacebookAI_roberta-base_r8 - Acc: 0.9945, Prec: 0.9952, Rec: 0.9943, F1: 0.9948, ROC-AUC: 0.9989
22:48:00 | INFO | Saved evaluation results to model_comparison.csv

MODEL COMPARISON - TEST SET RESULTS
                               model  accuracy  precision   recall  f1_score  roc_auc   pr_auc  training_time_min  trainable_params  total_params
google_electra-base-discriminator_r4  0.993889   0.994531 0.993851  0.994191 0.999311 0.999237         140.538083           1255682     110739460
google_electra-base-discriminator_r8  0.993529   0.995204 0.992485  0.993843 0.999258 0.999168         140.275087           1919234     111403012
          FacebookAI_roberta-base_r4  0.994128   0.994984 0.993851  0.994417 0.999009 0.998843         544.596072           1255682     125902852
          FacebookAI_roberta-base_r8  0.994488   0.995213 0.994307  0.994760 0.998905 0.998683         544.919773           1919234     126566404


In [ ]:
logger.info("Creating individual evaluation visualizations...")

for result in evaluation_results:
    run_name = result['model']

    fig, axes = plt.subplots(2, 2, figsize=tuple(config['visualization']['figure_sizes']['large']))

    cm = result['confusion_matrix']
    cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[0, 0], xticklabels=['Ham', 'Spam'], yticklabels=['Ham', 'Spam'])
    axes[0, 0].set_ylabel('True Label')
    axes[0, 0].set_xlabel('Predicted Label')
    axes[0, 0].set_title(f'{run_name} - Confusion Matrix (Counts)')

    sns.heatmap(cm_normalized, annot=True, fmt='.2%', cmap='Blues', ax=axes[0, 1], xticklabels=['Ham', 'Spam'], yticklabels=['Ham', 'Spam'])
    axes[0, 1].set_ylabel('True Label')
    axes[0, 1].set_xlabel('Predicted Label')
    axes[0, 1].set_title(f'{run_name} - Confusion Matrix (Normalized)')

    axes[1, 0].plot(result['fpr'], result['tpr'], linewidth=2, label=f'ROC (AUC = {result["roc_auc"]:.3f})')
    axes[1, 0].plot([0, 1], [0, 1], 'k--', linewidth=1)
    axes[1, 0].set_xlabel('False Positive Rate')
    axes[1, 0].set_ylabel('True Positive Rate')
    axes[1, 0].set_title(f'{run_name} - ROC Curve')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)

    axes[1, 1].plot(result['recall_curve'], result['precision_curve'], linewidth=2, label=f'PR (AUC = {result["pr_auc"]:.3f})')
    axes[1, 1].set_xlabel('Recall')
    axes[1, 1].set_ylabel('Precision')
    axes[1, 1].set_title(f'{run_name} - Precision-Recall Curve')
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3)

    plt.tight_layout()
    save_plot(f'{run_name}_evaluation.png', subdir='results')

    print(f"\n{run_name} Classification Report:")
    print(classification_report(result['true_labels'], result['predictions'], target_names=['Ham', 'Spam']))

22:48:00 | INFO | Creating individual evaluation visualizations...
22:48:02 | INFO | Saved plot: visualizations/results/google_electra-base-discriminator_r4_evaluation.png

google_electra-base-discriminator_r4 Classification Report:
              precision    recall  f1-score   support

         Ham       0.99      0.99      0.99      3954
        Spam       0.99      0.99      0.99      4391

    accuracy                           0.99      8345
   macro avg       0.99      0.99      0.99      8345
weighted avg       0.99      0.99      0.99      8345

22:48:03 | INFO | Saved plot: visualizations/results/google_electra-base-discriminator_r8_evaluation.png

google_electra-base-discriminator_r8 Classification Report:
              precision    recall  f1-score   support

         Ham       0.99      0.99      0.99      3954
        Spam       1.00      0.99      0.99      4391

    accuracy                           0.99      8345
   macro avg       0.99      0.99      0.99      8345
we

In [ ]:
logger.info("Creating comparison visualizations...")

fig, axes = plt.subplots(2, 3, figsize=tuple(config['visualization']['figure_sizes']['large']))

metrics = ['accuracy', 'precision', 'recall', 'f1_score', 'roc_auc', 'pr_auc']
metric_names = ['Accuracy', 'Precision', 'Recall', 'F1 Score', 'ROC-AUC', 'PR-AUC']

for idx, (metric, name) in enumerate(zip(metrics, metric_names)):
    row = idx // 3
    col = idx % 3

    ax = axes[row, col]
    values = results_df[metric].values
    models = results_df['model'].values

    colors = []
    for model in models:
        if 'google' in model:
            colors.append('#2E5EAA' if 'r8' in model else '#5A8FCC')
        elif 'roberta' in model:
            colors.append('#2D8E4D' if 'r8' in model else '#52B36F')
        else:
            colors.append('#E67E22' if 'r8' in model else '#F39C44')

    bars = ax.bar(range(len(models)), values, color=colors)
    ax.set_xticks(range(len(models)))
    ax.set_xticklabels(models, rotation=45, ha='right')
    ax.set_ylabel(name)
    ax.set_title(f'{name} Comparison')
    ax.grid(True, alpha=0.3, axis='y')

    for i, v in enumerate(values):
        ax.text(i, v + 0.01, f'{v:.3f}', ha='center', va='bottom', fontsize=8)

plt.tight_layout()
save_plot('metrics_comparison.png', subdir='results')

fig, ax = plt.subplots(figsize=tuple(config['visualization']['figure_sizes']['large']))

for result in evaluation_results:
    model_base = result['model'].split('_')[0]
    lora_r = result['model'].split('_')[1]
    linestyle = '-' if 'r8' in lora_r else '--'

    if 'google' in model_base:
        color = '#2E5EAA'
    elif 'roberta' in model_base:
        color = '#2D8E4D'
    else:
        color = '#E67E22'

    ax.plot(result['fpr'], result['tpr'], linestyle=linestyle, linewidth=2, color=color, label=f"{result['model']} (AUC={result['roc_auc']:.3f})")

ax.plot([0, 1], [0, 1], 'k--', linewidth=1, label='Random Classifier')
ax.set_xlabel('False Positive Rate', fontsize=12)
ax.set_ylabel('True Positive Rate', fontsize=12)
ax.set_title('ROC Curves - All Models Comparison', fontsize=14, fontweight='bold')
ax.legend(loc='lower right')
ax.grid(True, alpha=0.3)

plt.tight_layout()
save_plot('roc_curves_comparison.png', subdir='results')

fig, ax = plt.subplots(figsize=tuple(config['visualization']['figure_sizes']['large']))

for result in evaluation_results:
    model_base = result['model'].split('_')[0]
    lora_r = result['model'].split('_')[1]
    linestyle = '-' if 'r8' in lora_r else '--'

    if 'google' in model_base:
        color = '#2E5EAA'
    elif 'roberta' in model_base:
        color = '#2D8E4D'
    else:
        color = '#E67E22'

    ax.plot(result['recall_curve'], result['precision_curve'], linestyle=linestyle,
            linewidth=2, color=color, label=f"{result['model']} (AUC={result['pr_auc']:.3f})")

ax.set_xlabel('Recall', fontsize=12)
ax.set_ylabel('Precision', fontsize=12)
ax.set_title('Precision-Recall Curves - All Models Comparison', fontsize=14, fontweight='bold')
ax.legend(loc='lower left')
ax.grid(True, alpha=0.3)

plt.tight_layout()
save_plot('pr_curves_comparison.png', subdir='results')

22:48:05 | INFO | Creating comparison visualizations...
22:48:06 | INFO | Saved plot: visualizations/results/metrics_comparison.png
22:48:07 | INFO | Saved plot: visualizations/results/roc_curves_comparison.png
22:48:07 | INFO | Saved plot: visualizations/results/pr_curves_comparison.png


'visualizations/results/pr_curves_comparison.png'

In [ ]:
logger.info("Performing LoRA ablation analysis...")

base_models = set([r['model'].rsplit('_', 1)[0] for r in evaluation_results])

ablation_data = []
for base_model in base_models:
    r4_result = next((r for r in evaluation_results if r['model'] == f"{base_model}_r4"), None)
    r8_result = next((r for r in evaluation_results if r['model'] == f"{base_model}_r8"), None)

    if r4_result and r8_result:
        ablation_data.append({
            'model': base_model,
            'r4_f1': r4_result['f1_score'],
            'r8_f1': r8_result['f1_score'],
            'f1_diff': r8_result['f1_score'] - r4_result['f1_score'],
            'r4_precision': r4_result['precision'],
            'r8_precision': r8_result['precision'],
            'r4_recall': r4_result['recall'],
            'r8_recall': r8_result['recall'],
            'r4_params': r4_result['trainable_params'],
            'r8_params': r8_result['trainable_params'],
            'params_ratio': r8_result['trainable_params'] / r4_result['trainable_params']
        })

ablation_df = pd.DataFrame(ablation_data)
ablation_df.to_csv(Path(config['paths']['results']) / 'lora_ablation.csv', index=False)

print("\n" + "="*60)
print("LoRA ABLATION STUDY (r=4 vs r=8)")
print("="*60)
print(ablation_df.to_string(index=False))

fig, axes = plt.subplots(1, 3, figsize=tuple(config['visualization']['figure_sizes']['large']))

x = np.arange(len(ablation_df))
width = 0.35

axes[0].bar(x - width/2, ablation_df['r4_f1'], width, label='r=4', alpha=0.8)
axes[0].bar(x + width/2, ablation_df['r8_f1'], width, label='r=8', alpha=0.8)
axes[0].set_xlabel('Model')
axes[0].set_ylabel('F1 Score')
axes[0].set_title('F1 Score: LoRA r=4 vs r=8')
axes[0].set_xticks(x)
axes[0].set_xticklabels(ablation_df['model'], rotation=45, ha='right')
axes[0].legend()
axes[0].grid(True, alpha=0.3, axis='y')

axes[1].bar(ablation_df['model'], ablation_df['f1_diff'], color=['green' if d > 0 else 'red' for d in ablation_df['f1_diff']])
axes[1].axhline(y=0, color='black', linestyle='-', linewidth=0.5)
axes[1].set_xlabel('Model')
axes[1].set_ylabel('F1 Difference (r=8 - r=4)')
axes[1].set_title('Performance Gain from r=8')
axes[1].tick_params(axis='x', rotation=45)
axes[1].grid(True, alpha=0.3, axis='y')

axes[2].bar(ablation_df['model'], ablation_df['params_ratio'])
axes[2].axhline(y=1, color='red', linestyle='--', linewidth=1)
axes[2].set_xlabel('Model')
axes[2].set_ylabel('Parameter Ratio (r=8 / r=4)')
axes[2].set_title('Trainable Parameters Increase')
axes[2].tick_params(axis='x', rotation=45)
axes[2].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
save_plot('lora_ablation_analysis.png', subdir='results')

22:52:25 | INFO | Performing LoRA ablation analysis...

LoRA ABLATION STUDY (r=4 vs r=8)
                            model    r4_f1    r8_f1   f1_diff  r4_precision  r8_precision  r4_recall  r8_recall  r4_params  r8_params  params_ratio
          FacebookAI_roberta-base 0.994417 0.994760  0.000342      0.994984      0.995213   0.993851   0.994307    1255682    1919234       1.52844
google_electra-base-discriminator 0.994191 0.993843 -0.000348      0.994531      0.995204   0.993851   0.992485    1255682    1919234       1.52844
22:52:26 | INFO | Saved plot: visualizations/results/lora_ablation_analysis.png


'visualizations/results/lora_ablation_analysis.png'

# 8. Conclusion & Key Findings

## Results Summary

Based on comprehensive evaluation across 4 model configurations:

**Best Performing Model:**
- **RoBERTa-base with LoRA rank 8** achieved the highest test accuracy (99.45%) and F1 (99.48%).
- **Note:** All four models achieved highly competitive F1 scores (~99.38%).

**Key Insights:**
1. **LoRA Efficiency**: All models achieved over 99% accuracy with only 1.2M to 1.9M trainable parameters (a massive efficiency gain over full fine-tuning).
2. **Speed vs. Accuracy Trade-off**: The top-performing RoBERTa-r8 is significantly slower in inference (9.4 minutes) compared to ELECTRA-r4 (3.6 minutes).
3. **Optimized Choice**: **google_electra-base-discriminator_r4** delivers a nearly identical F1 score (99.42%) but with the **fastest inference time** and the **lowest trainable parameter count** (1.2M).
4. **Rank Comparison**: Rank 8 provided only marginal F1 improvements (~0.04-0.06% F1) over Rank 4, making Rank 4 the more parameter-efficient choice.

**Production Recommendation:**
**google_electra-base-discriminator_r4** offers the optimal balance of high performance (F1 99.42%) and operational efficiency (fastest inference, lowest trainable parameters) for deployment.

**Next Steps:**
- Deploy the **google_electra-base-discriminator_r4** model with threshold tuning for production precision targets
- Monitor performance on live data for concept drift
- Periodic retraining as spam tactics evolve

In [ ]:
dir()

['In',
 'Out',
 '_',
 '__',
 '___',
 '__builtin__',
 '__builtins__',
 '__doc__',
 '__loader__',
 '__name__',
 '__package__',
 '__session__',
 '__spec__',
 '_dh',
 '_i',
 '_i1',
 '_ih',
 '_ii',
 '_iii',
 '_oh',
 'exit',
 'get_ipython',
 'open',
 'quit']